### Load dataset

In [14]:
import pandas as pd
from enum import StrEnum, auto
RANDOM_STATE = 42

# NOTE: StrEnum requires Python 3.11+
#       Can refactor to CONSTANTS instead
class FeatureVariant(StrEnum):
    LITERATURE = auto()
    RESEARCH_HUMAN = 'researchHumanAssessed'
    RESEARCH_BORUTA = 'researchBORUTA'
    STATISTICAL = auto()
    AUTOMATED = auto()

# Can be replaced with desired variant for different feature sets
GENE_FILE_VARIANT = FeatureVariant.RESEARCH_HUMAN

FILE_PATH = f"../Data/patient_genes_{GENE_FILE_VARIANT}.csv"

variant = 'svm'
df = pd.read_csv(FILE_PATH)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 977 entries, 0 to 976
Data columns (total 33 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   FYN      977 non-null    int64 
 1   BRCA1    977 non-null    int64 
 2   FOXC1    977 non-null    int64 
 3   TBC1D1   977 non-null    int64 
 4   LAG3     977 non-null    int64 
 5   CDK6     977 non-null    int64 
 6   GATA3    977 non-null    int64 
 7   CCND1    977 non-null    int64 
 8   PRR4     977 non-null    int64 
 9   EPCAM    977 non-null    int64 
 10  CD274    977 non-null    int64 
 11  PIK3CA   977 non-null    int64 
 12  TOP2A    977 non-null    int64 
 13  DCLK1    977 non-null    int64 
 14  MYC      977 non-null    int64 
 15  LRPPRC   977 non-null    int64 
 16  BRCA2    977 non-null    int64 
 17  TP53     977 non-null    int64 
 18  EGFR     977 non-null    int64 
 19  MKI67    977 non-null    int64 
 20  TTN      977 non-null    int64 
 21  CTLA4    977 non-null    int64 
 22  PT

### Import functions to assess data

In [16]:
from sklearn.svm import SVC

%run "DataHelpers.ipynb"

### Dataset split: training and test data

In [17]:
X, y, X_train, X_test, y_train, y_test, test_case_ids = split_data(df, "tnbc", True)
print("\nApplied Smote")
X_smote, y_smote, X_train_smote, X_test_smote, y_train_smote, y_test_smote, test_case_ids_smote = split_data_apply_smote(df, "tnbc")

X_train.shape=(781, 31)
X_test.shape=(196, 31)
y_train.shape=(781,)
y_test.shape=(196,)

Applied Smote
X_train.shape=(1379, 31)
X_test.shape=(345, 31)
y_train.shape=(1379,)
y_test.shape=(345,)


### Support Vector Machine (SVM)

In [18]:
# Create model

model = SVC(random_state=RANDOM_STATE, probability=True)

def run_model(X_train: pd.DataFrame, X_test: pd.DataFrame, y_train: pd.Series, y_test: pd.Series, test_case_ids: pd.Series, is_smote: bool):
    # Train the model
    model.fit(X_train, y_train)

    # Model predictions
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]  # For ROC curves etc.

    # Save it in a dataframe, to CSV
    predictions = pd.DataFrame({
        "case_id": test_case_ids,
        "y_test": y_test,
        "y_pred": y_pred,
        "y_prob": y_prob
    })
    predictions.to_csv(f"../Data/model_output_{variant}_{GENE_FILE_VARIANT}_{'smote' if is_smote else ''}.csv", index=False)

    return y_pred, y_prob

In [19]:
y_pred, y_prod = run_model(X_train, X_test, y_train, y_test, test_case_ids, False)
print_evaluated_model_accuracy(y_test, y_pred)

Accuracy: 0.94


## Smote applied

In [20]:
y_pred_smote, y_prod_smote = run_model(X_train_smote, X_test_smote, y_train_smote, y_test_smote, test_case_ids_smote, True)

print_evaluated_model_accuracy(y_test_smote, y_pred_smote)

Accuracy: 0.94


## Model cross validation

In [21]:
def run_cross_validation(X: pd.DataFrame, y: pd.Series, y_test: pd.Series, y_pred: pd.Series, y_prob: pd.Series, is_smote: bool) -> pd.DataFrame:
    metrics: pd.DataFrame = get_cross_validation_metrics(model, X, y, cv=5)
    test_metrics = get_metrics(y_test, y_pred, y_prob)
    test_metrics["fold"] = 0 # Initial test metrics (before cross validation)
    test = pd.DataFrame([test_metrics])
    test.set_index("fold", inplace=True)

    print_validated_model_accuracy(model, metrics)

    # Prepend test_metrics to metrics dataframe, export and display
    metrics = pd.concat([test, metrics])
    metrics.to_csv(f"../Data/model_metrics_{variant}_{GENE_FILE_VARIANT}{'_smote' if is_smote else ''}.csv", index=False)
    return metrics

In [22]:
metrics = run_cross_validation(X, y, y_test, y_pred, y_prod, False)
metrics

Model validation for SVC:
[0.9336734693877551, 0.9285714285714286, 0.9333333333333333, 0.9128205128205128, 0.9076923076923077]

Mean accuracy: 0.9232



,accuracy,recall,precision,f1_score,roc_auc,true_positive,true_negative,false_positive,false_negative
fold,,,,,,,,,
0,0.943878,0.652174,0.833333,0.731707,0.965067,15,170,3,8
1,0.933673,0.608696,0.777778,0.682927,0.970847,14,169,4,9
2,0.928571,0.608696,0.736842,0.666667,0.957276,14,168,5,9
3,0.933333,0.521739,0.857143,0.648649,0.964863,12,170,2,11
4,0.912821,0.521739,0.666667,0.585366,0.968655,12,166,6,11
5,0.907692,0.478261,0.647059,0.550000,0.927199,11,166,6,12


In [24]:
metric_smote = run_cross_validation(X_smote, y_smote, y_test_smote, y_pred_smote, y_prod_smote, True)
metric_smote

Model validation for SVC:
[0.9333333333333333, 0.936231884057971, 0.9188405797101449, 0.936231884057971, 0.9273255813953488]

Mean accuracy: 0.9304



,accuracy,recall,precision,f1_score,roc_auc,true_positive,true_negative,false_positive,false_negative
fold,,,,,,,,,
0,0.942029,0.976744,0.913043,0.943820,0.975434,168,157,16,4
1,0.933333,0.988372,0.890052,0.936639,0.967704,170,152,21,2
2,0.936232,0.976744,0.903226,0.938547,0.969653,168,155,18,4
3,0.918841,0.936416,0.905028,0.920455,0.972443,162,155,17,11
4,0.936232,0.953757,0.921788,0.937500,0.962562,165,158,14,8
5,0.927326,0.988372,0.880829,0.931507,0.969240,170,149,23,2
